## Sentiment Analysis Challenge
#### Naive Bayes Implementation
For my "from scratch" implementation I've done fairly basic naive bayes. I attempted to add
things that would account for more complex features such as data cleaning and word pair
analysis but, surprisingly, these reduced accuracy significantly. I'll talk more about those
experiments throughout the notebook.

Imports

In [155]:
import csv
import math

## Set Input Variables Here
**To input test data, simply add it to the data directory and set `test_file` to the relative
filepath. If `test_file` has a value, `validation_percent` will be ignored and no validation
set will be used.**

Set `using_word_pairs` to True to add word pair analysis. This essentially creates
a word pair "bag of words" that is also factored into the prediction. With the Welsh tweets,
this performs significantly worse. After playing around with Google translate for a bit, my
theory as to why this is two fold.

For one, certain Welsh words that are highly connected don't
necessarily appear together. For example, a sentence that can easily fool basic naive
bayes is "I'm not happy". Happy definitely has a very high positive probability and without
word pairs "not happy" cannot be recognized as highly negative. However, in Welsh, "not" is
"ddim" and "happy" is "hapus". However "I'm not happy" is "Dwi ddim yn hapus". The words
for "not" and "happy" don't appear next to each other as they do in English, so triple word
combinations (or more) would have to be used to recognize this in Welsh.
I considered translating each tweet to English and then running the word pair analysis but
could not do so considering this is the "from scratch" rather than the "anything goes"
implementation.

Secondly, I don't think the dataset is large enough to statistically recognize if a string of 2+
words is positive or negative (won't have enough non-unique combinations) with something
as simple as naive bayes.

**Note: for highest accuracy on the Welsh language tweets dataset, make sure `using_word_pairs` is set to `False`**

In [156]:
train_file = 'data/train.tsv'
validation_percent = 0.1
test_file = ''
using_word_pairs = False

Import the data from provided file, only including valid rows with both labels and data.
There were a few inputs that did not have labels so this has a simple data cleaning check
to confirm both a label and a document are present before adding.

In [157]:
print("Loading training data...")
labels = []
inputs = []
with open(train_file, encoding='utf-8') as data:
  reader = csv.reader(data, delimiter='\t')
  for row in reader:
    if len(row) == 2:
        labels.append(row[0])
        inputs.append(row[1])
print("Loaded {} documents".format(len(labels)))

Loading training data...
Loaded 78608 documents


This does the same as above for the test file if one is provided. If not, it splits the training
data into a train set and validation split based on the split provided in the *Inputs* cell.

In [158]:
if len(test_file) > 0:
    print("Loading testing data...")
    with open(test_file, encoding='utf-8') as data:
      reader = csv.reader(data, delimiter='\t')
      test_labels = []
      test_inputs = []
      for row in reader:
        if len(row) == 2:
            test_labels.append(row[0])
            test_inputs.append(row[1])
    print("Loaded {} documents".format(len(test_labels)))
    train_labels = labels
    train_inputs = inputs
else:
    print("Splitting training and validation sets...")
    num_train_samples = int(len(labels) * (1-validation_percent))
    train_labels = labels[:num_train_samples]
    train_inputs = inputs[:num_train_samples]
    test_labels = labels[num_train_samples:]
    test_inputs = inputs[num_train_samples:]
    print("Training set has {} documents".format(len(train_labels)))
    print("Validation set has {} documents".format(len(test_labels)))

Splitting training and validation sets...
Training set has 70747 documents
Validation set has 7861 documents


The `count_words` function counts the number of positive and negative occurrences of each
word and returns the counts in dictionary form.

I tested punctuation removal and lowercase normalizing here but both performed
significantly (at least 3%) worse on validation data. I suspect this is because things like
exclamation marks or all caps are reasonably accurate predictors of sentiment, especially
binary positive vs negative sentiment.

In [159]:
def count_words(documents_list):
    print("Counting words...")
    _counts = {'positive': {}, 'negative': {}}
    for _idx, document in enumerate(documents_list):
            _sentiment = 'positive' if int(labels[_idx]) else 'negative'
            for word in document.split():
                if word in _counts[_sentiment]:
                    _counts[_sentiment][word] += 1
                else:
                    _counts[_sentiment][word] = 1
    print("Counted {} positive, {} negative words".format(len(_counts['positive']), len(_counts['negative'])))
    return _counts

This function is the same as above except it counts word pairs (each word and the word following).
This won't be used if `using_word_pairs` is set to `False`

In [160]:
def count_word_pairs(documents_list):
    print("Counting word pairs...")
    _counts = {'positive': {}, 'negative': {}}
    for _idx, document in enumerate(documents_list):
            _sentiment = 'positive' if int(labels[_idx]) else 'negative'
            _word_list = document.split()
            for word_place, word in enumerate(_word_list):
                if word_place < len(_word_list)-1:
                    word_pair = word + ' ' + _word_list[word_place+1]
                    if word_pair in _counts[_sentiment]:
                        _counts[_sentiment][word_pair] += 1
                    else:
                        _counts[_sentiment][word_pair] = 1
    print("Counted {} positive, {} negative word pairs".format(len(_counts['positive']), len(_counts['negative'])))
    return _counts

`fit` calculates the probabilities of a word (or word pair) being present given the class.
As per the Naive Bayes setup in the textbook it's based on the word count and size of each class's "bag of words".
While I include Laplace smoothing below if a word hasn't been encountered yet, I don't add 1 here
simply because it caused significantly worse performance in my experiments. I'm not entirely sure why this is.

In [161]:
def fit_words(_inputs):
    _probabilities = count_words(_inputs)
    num_positive = len(_probabilities['positive'])
    num_negative = len(_probabilities['negative'])
    for pos_word in _probabilities['positive']:
        _probabilities['positive'][pos_word] = math.log(_probabilities['positive'][pos_word] / num_positive)
    for neg_word in _probabilities['negative']:
        _probabilities['negative'][neg_word] = math.log(_probabilities['negative'][neg_word] / num_negative)
    return _probabilities

def fit_pairs(_inputs):
    _pair_probabilities = count_word_pairs(_inputs)
    num_positive = len(_pair_probabilities['positive'])
    num_negative = len(_pair_probabilities['negative'])
    for pos_pair in _pair_probabilities['positive']:
        _pair_probabilities['positive'][pos_pair] = math.log(_pair_probabilities['positive'][pos_pair] / num_positive)
    for neg_pair in _pair_probabilities['negative']:
        _pair_probabilities['negative'][neg_pair] = math.log(_pair_probabilities['negative'][neg_pair] / num_negative)
    return _pair_probabilities

def fit(_inputs):
    print("Computing probabilities...")
    return fit_words(_inputs), fit_pairs(_inputs)

word_probabilities, pair_probabilities = fit(train_inputs)

Computing probabilities...
Counting words...
Counted 75833 positive, 72061 negative words
Counting word pairs...
Counted 299951 positive, 290987 negative word pairs


These functions are simply to retrieve the probabilities if a word has been seen before,
and to return 1 / the word count (Laplace smoothing) if not.

In [162]:
def get_positive_word_probability(_word):
    if _word in word_probabilities['positive']:
        return word_probabilities['positive'][_word]
    else:
        return math.log(1 / len(word_probabilities['positive']))
def get_negative_word_probability(_word):
    if _word in word_probabilities['negative']:
        return word_probabilities['negative'][_word]
    else:
        return math.log(1 / len(word_probabilities['negative']))
def get_positive_pair_probability(_pair):
    if _pair in pair_probabilities['positive']:
        return pair_probabilities['positive'][_pair]
    else:
        return math.log(1 / len(word_probabilities['positive']))
def get_negative_pair_probability(_pair):
    if _pair in pair_probabilities['negative']:
        return pair_probabilities['negative'][_pair]
    else:
        return math.log(1 / len(pair_probabilities['negative']))

`predict` adds the positive probabilities and the negative probabilities and compares, returning the higher
probability as the prediction.

In [163]:
def predict(_document):
    positive_sum = 0
    negative_sum = 0
    _word_list = _document.split()
    for _idx, _word in enumerate(_word_list):
        positive_sum += get_positive_word_probability(_word)
        negative_sum += get_negative_word_probability(_word)
        if using_word_pairs and _idx < len(_word_list)-1:
            positive_sum += get_positive_pair_probability(_word + ' ' + _word_list[_idx+1])
            negative_sum += get_negative_pair_probability(_word + ' ' + _word_list[_idx+1])
    return 1 if positive_sum > negative_sum else 0

This runs the algorithm and returns the validation/test accuracy.

In [164]:
correct = 0
print("Running...")
for idx, doc in enumerate(test_inputs):
    if predict(doc) == int(test_labels[idx]):
        correct += 1
print("Done!")
val_or_test = "Validation" if len(test_file) <= 0 else "Testing"
print("{} Accuracy:".format(val_or_test), str(round(correct / len(test_inputs) * 100, 2))+"%")

Running...
Done!
Validation Accuracy: 73.54%
